In [ ]:

!pip install --upgrade pip
!pip install tensorflow
!pip install codecarbon
!pip install opencv-python pandas numpy tqdm


In [ ]:
 import os, time, pathlib, math, random

from pathlib import Path

import numpy as np

import matplotlib.pyplot as plt

import pandas as pd
 
import tensorflow as tf

from tensorflow import keras

from keras.models import Sequential

from keras.utils import to_categorical

from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten

from tensorflow.keras.preprocessing.image import ImageDataGenerator
 
import cv2

from codecarbon import EmissionsTracker
 
# ---------- Config ----------

SEED = 42

random.seed(SEED); np.random.seed(SEED); tf.random.set_seed(SEED)
 
ROOT = Path(".")

RESULTS_CSV = ROOT / "results.csv"

LOGDIR = (ROOT / "codecarbon_logs").resolve()

LOGDIR.mkdir(parents=True, exist_ok=True)
 
CODECARBON_KWARGS = dict(

    measure_power_secs=1,

    save_to_file=True,

    output_dir=str(LOGDIR),

    log_level="warning",

    tracking_mode="process",  

)
 
print(f"[CodeCarbon] logs → {LOGDIR}")
 

def is_blurry(image, threshold=100):

    """

    Returns True if image is blurry based on Laplacian variance.

    CIFAR-10 is RGB, so use COLOR_RGB2GRAY.

    """

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

    variance = cv2.Laplacian(gray, cv2.CV_64F).var()

    return variance < threshold
 
def check_blur(images, threshold=100):

    blurry_images = []

    for i, image in enumerate(images):

        if is_blurry(image, threshold):

            blurry_images.append(i)

    return blurry_images
 
def is_black(image, threshold=10):

    """

    Returns True if image is near-black.

    Threshold is the mean pixel intensity (0-255 scale).

    """

    return float(np.mean(image)) < threshold
 
def check_black(images, threshold=10):

    black_indices = []

    for i, image in enumerate(images):

        if is_black(image, threshold):

            black_indices.append(i)

    return black_indices
 
def display_images_with_indices(images, indices, cols=8, title="Selected images"):

    if len(indices) == 0:

        print("[display] No images to display.")

        return

    rows = int(math.ceil(len(indices) / cols))

    plt.figure(figsize=(cols*2, rows*2))

    for i, idx in enumerate(indices[: rows*cols]):

        plt.subplot(rows, cols, i+1)

        plt.imshow(images[idx])

        plt.title(f"idx={idx}", fontsize=8)

        plt.axis("off")

    plt.suptitle(title)

    plt.show()
 
# ---------- Phase helpers ----------

def run_with_tracker(phase_name, fn):

    tracker = EmissionsTracker(project_name=phase_name, **CODECARBON_KWARGS)

    tracker.start()

    t0 = time.time()

    try:

        out = fn()

    finally:

        emissions = tracker.stop() or 0.0

    secs = time.time() - t0

    print(f"[{phase_name}] time={secs:.2f}s  emissions={emissions:.6f} kg")

    return out, secs, emissions
 
# ============================================

# 1) PREPROCESS PHASE

# ============================================

def preprocess_phase():

    # Load CIFAR-10

    (train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()

    print('Training data shape : ', train_images.shape, train_labels.shape)

    print('Testing  data shape : ', test_images.shape,  test_labels.shape)

    classes = np.unique(train_labels)

    nClasses = len(classes)

    print('Total number of outputs : ', nClasses)

    print('Output classes : ', classes.reshape(-1))
 
    # Clean: remove blurry & black from TRAIN 

    BLUR_THR  = 100.0

    BLACK_THR = 10.0
 
    blurry_idx = check_blur(train_images, threshold=BLUR_THR)

    black_idx  = check_black(train_images, threshold=BLACK_THR)

    print(f"Found blurry={len(blurry_idx)}  black={len(black_idx)} in train")
 

    to_remove = sorted(set(blurry_idx) | set(black_idx))

    if to_remove:

        keep_mask = np.ones(len(train_images), dtype=bool)

        keep_mask[to_remove] = False

        train_images = train_images[keep_mask]

        train_labels = train_labels[keep_mask]

        print(f"Removed {len(to_remove)} images → train now {train_images.shape}")
 
    # Scale to [0,1]

    train_images = train_images.astype("float32") / 255.0

    test_images  = test_images.astype("float32")  / 255.0
 
    # One-hot labels

    y_train = to_categorical(train_labels, nClasses)

    y_test  = to_categorical(test_labels,  nClasses)
 
    # Augmentation (light)

    aug = ImageDataGenerator(

        horizontal_flip=True,

    )

    aug.fit(train_images)
 
    return (train_images, y_train), (test_images, y_test), nClasses, aug
 
(prep_out, preprocess_secs, preprocess_em) = run_with_tracker("preprocess", preprocess_phase)

print(" fin preprocessing")

(train_images, y_train), (test_images, y_test), nClasses, aug = prep_out
 
# ============================================

# 2) BUILD + TRAIN PHASE (Keras Sequential CNN)

# ============================================

def train_phase():

    model = Sequential([

        Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(32,32,3), padding="same"),

        MaxPooling2D(pool_size=(2,2)),

        Dropout(0.25),
 
        Conv2D(64, kernel_size=(3,3), activation='relu', padding="same"),

        MaxPooling2D(pool_size=(2,2)),

        Dropout(0.25),
 
        Conv2D(128, kernel_size=(3,3), activation='relu', padding="same"),

        MaxPooling2D(pool_size=(2,2)),

        Dropout(0.25),
 
        Flatten(),

        Dense(256, activation='relu'),

        Dropout(0.5),

        Dense(nClasses, activation='softmax')

    ])
 
    model.compile(optimizer=keras.optimizers.Adam(1e-3),

                  loss='categorical_crossentropy',

                  metrics=['accuracy'])
 
    BATCH = 64

    EPOCHS = 50
 
    # Use augmented batches for training; evaluate on clean test

    history = model.fit(

        aug.flow(train_images, y_train, batch_size=BATCH, shuffle=True),

        epochs=EPOCHS,

        validation_data=(test_images, y_test),

        verbose=1

    )

    params = model.count_params()

    return model, params, history.history
 
((model, params, history), train_secs, train_em) = run_with_tracker("train", train_phase)
 
# ============================================

# 3) EVALUATION PHASE

# ============================================

def eval_phase():

    loss, acc = model.evaluate(test_images, y_test, verbose=0)

    print(f"Test accuracy: {acc:.4f}")

    return float(acc)
 
(test_acc, eval_secs, eval_em) = run_with_tracker("evaluate", eval_phase)
 
# ============================================

# 4) Save / append results

# ============================================

row = {

    "model": "Keras-Sequential-CNN(clean:blur+black + flip aug)",

    "params": params,

    "test_acc": round(test_acc, 4),

    "preprocess_secs": round(preprocess_secs, 2),

    "train_secs": round(train_secs, 2),

    "eval_secs": round(eval_secs, 2),

    "preprocess_emissions_kg": preprocess_em,

    "train_emissions_kg": train_em,

    "eval_emissions_kg": eval_em,

    "total_secs": round(preprocess_secs + train_secs + eval_secs, 2),

    "total_emissions_kg": preprocess_em + train_em + eval_em,

}

df_new = pd.DataFrame([row])
 
if RESULTS_CSV.exists():

    try:

        df_old = pd.read_csv(RESULTS_CSV)

        df_out = pd.concat([df_old, df_new], ignore_index=True)

    except Exception:

        df_out = df_new

else:

    df_out = df_new
 
df_out.to_csv(RESULTS_CSV, index=False)

print("Appended results →", RESULTS_CSV.resolve())

df_new